In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
from IPython.display import display, Markdown
import base64
from PIL import Image
import io
import json
import subprocess

In [ ]:
def get_video_entities(offset=0, limit=10):
    endpoint = "http://localhost:7200/repositories/kgrc4si"
    sparql = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX mssn: <http://mssn.sigappfr.org/mssn/>
    PREFIX vh2kg: <http://kgrc4si.home.kg/virtualhome2kg/ontology/>
    PREFIX ex: <http://kgrc4si.home.kg/virtualhome2kg/instance/>
    PREFIX ac: <http://kgrc4si.home.kg/virtualhome2kg/ontology/action/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX sosa: <http://www.w3.org/ns/sosa/>
    select distinct ?activity ?event ?prevObjectName ?action ?object ?frame ?targetObject
    where {
        ?prevEvent vh2kg:hasVideoSegment ?visualSegment .
        ?visualSegment mssn:hasMediaDescriptor ?frame_r ;
            vh2kg:hasEndFrame ?endFrame .
        ?frame_r vh2kg:frameNumber ?frame .
        filter(?frame < ?endFrame && ?frame >= (?endFrame - 5))
        ?frame_r mssn:hasMediaDescriptor ?bbox2d_1 .
        ?bbox2d_1 vh2kg:is2DbboxOf ?prevObject ;
                rdfs:label ?prevObjectName .
        
        ?activity vh2kg:hasEvent ?event .
        ?prevEvent vh2kg:action ac:grab ;
            vh2kg:mainObject ?prevObject ;
            vh2kg:nextEvent ?prevEvent2 .
        ?prevEvent2 vh2kg:action ac:walk ;
            vh2kg:nextEvent ?event .
        ?event vh2kg:action ?action ;
            vh2kg:mainObject ?object .
        optional { ?event vh2kg:targetObject ?targetObject }
        
        BIND(RAND() AS ?rand)
    } order by ?rand offset """ + str(offset) +  """ limit """ + str(limit) + """
    """
    wrapper = SPARQLWrapper(endpoint)
    wrapper.setQuery(sparql)
    wrapper.setReturnFormat(JSON)
    results = wrapper.query().convert()
    return results

In [ ]:
def get_video_base64(activity_ent):
    endpoint = "http://localhost:7200/repositories/kgrc4si"
    sparql = """
        PREFIX vh2kg: <http://kgrc4si.home.kg/virtualhome2kg/ontology/>
        PREFIX sosa: <http://www.w3.org/ns/sosa/>
        PREFIX ex: <http://kgrc4si.home.kg/virtualhome2kg/instance/>
        SELECT ?base64 ?frameRate WHERE {
            ?video vh2kg:isVideoOf <""" + activity_ent + """> ;
        sosa:madeBySensor ?camera .
    ?video vh2kg:video ?base64 ;
           vh2kg:frameRate ?frameRate .
    filter(?camera != ex:camera2_scene1 && ?camera != ex:camera2_scene2 && ?camera != ex:camera2_scene3 && ?camera != ex:camera2_scene4 && ?camera != ex:camera2_scene5 && ?camera != ex:camera2_scene6 && ?camera != ex:camera2_scene7)
        } order by ?base64 limit 1
    """
    wrapper = SPARQLWrapper(endpoint)
    wrapper.setQuery(sparql)
    wrapper.setReturnFormat(JSON)
    results = wrapper.query().convert()
    return results

In [ ]:
import re
def remove_url_and_id(object_ent):
    object_name = object_ent.replace("http://kgrc4si.home.kg/virtualhome2kg/instance/", "")
    object_name = re.sub(r'\d+_scene\d', '', object_name)
    return object_name

In [ ]:
few_shot_size = 10
test_size = 100

results = get_video_entities(offset=0, limit=few_shot_size+test_size)

In [ ]:

bindings = results["results"]["bindings"]
image_list = []

few_shot_samples = []
ground_truth = []
i = 0
for result in bindings:
    activity_ent = result["activity"]["value"]
    # video_ent = result["video"]["value"]
    prev_object_name = result["prevObjectName"]["value"]
    action_ent = result["action"]["value"]
    action_name = action_ent.replace("http://kgrc4si.home.kg/virtualhome2kg/ontology/action/", "")
    object_ent = result["object"]["value"]
    object_name = remove_url_and_id(object_ent)
    target_object_ent = None
    target_object_name = None
    if "targetObject" in result:
        target_object_ent = result["targetObject"]["value"]
        target_object_name = remove_url_and_id(target_object_ent)
    
    answer = "The person will " + action_name
    if action_name == "sit":
        answer += " on"
    elif action_name == "walk":
        answer += " to"
    answer += " the " + object_name
    if target_object_ent is not None:
        if action_name == "put":
            answer += " on the " + target_object_name
        elif action_name == "pour":
            answer += " into the " + target_object_name
        else:
            answer += " to the " + target_object_name
    answer += "."
    
    # action_ent = result["action"]["value"]
    frame = result["frame"]["value"]
    video_results = get_video_base64(activity_ent)
    video_base64 = video_results["results"]["bindings"][0]["base64"]["value"]
    frame_rate = video_results["results"]["bindings"][0]["frameRate"]["value"]
    time = (float(frame))/float(frame_rate)


    # base64デコードしてバイナリデータを取得
    decoded_data = base64.b64decode(video_base64)

    # ファイルに書き込み
    with open("./tmp/decoded" + str(i) + ".mp4", "wb") as file:
        file.write(decoded_data)

    # Command to execute ffmpeg
    print(time)
    command = "ffmpeg -i ./tmp/decoded" + str(i) + ".mp4 -r 30 -ss " + str(time) + " -frames:v 1  ./test/" + str(i) + ".png"

    # Execute the command
    subprocess.run(command, shell=True)


    if i >= test_size:
         few_shot_samples.append({
            "activity_entity": activity_ent,
            "prev_object_name": prev_object_name, 
            "action": action_name,
            "object": object_name,
            "target_object": target_object_name,
            "frame": frame,
            "answer": answer
        })
    else:
        ground_truth.append({
            "activity_entity": activity_ent,
            "prev_object_name": prev_object_name, 
            "action": action_name,
            "object": object_name,
            "target_object": target_object_name,
            "frame": frame,
            "answer": answer
        })

    # Convert the ground_truth variable to a JSON list
    few_shot_samples_json = json.dumps(few_shot_samples)
    ground_truth_json = json.dumps(ground_truth)

    # Save the JSON list to a file
    with open('test/few_shot_samples.json', 'w') as file:
        file.write(few_shot_samples_json)

    with open('test/ground_truth.json', 'w') as file:
        file.write(ground_truth_json)

    i += 1